In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp

import numpy as np

In [2]:
tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
# Load data -- graph of a [cardioid](https://en.wikipedia.org/wiki/Cardioid).
n = 2000
t = tfd.Uniform(low=-np.pi, high=np.pi).sample([n, 1])
r = 2 * (1 - tf.cos(t))
x = r * tf.sin(t) + tfd.Normal(loc=0., scale=0.1).sample([n, 1])
y = r * tf.cos(t) + tfd.Normal(loc=0., scale=0.1).sample([n, 1])
# Model the distribution of y given x with a Mixture Density Network.
event_shape = [1]
num_components = 3
params_size = tfpl.MixtureLogistic.params_size(num_components, event_shape)
model = tfk.Sequential([
tfkl.Dense(12, activation='relu'),
tfkl.Dense(params_size, activation=None),
tfpl.MixtureLogistic(num_components, event_shape)
])
# Fit.
batch_size = 100
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.02),
            loss=lambda y, model: -model.log_prob(y))
model.fit(x, y,
        batch_size=batch_size,
        epochs=20,
        steps_per_epoch=n // batch_size)

2022-05-26 11:31:34.424385: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20


2022-05-26 11:31:34.826764: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


20/20 [==============================] - 1s 2ms/step - loss: 1.8397
Epoch 2/20
20/20 [==============================] - 0s 2ms/step - loss: 1.5736
Epoch 3/20
20/20 [==============================] - 0s 2ms/step - loss: 1.3091
Epoch 4/20
20/20 [==============================] - 0s 2ms/step - loss: 0.9618
Epoch 5/20
20/20 [==============================] - 0s 2ms/step - loss: 0.7171
Epoch 6/20
20/20 [==============================] - 0s 2ms/step - loss: 0.5798
Epoch 7/20
20/20 [==============================] - 0s 2ms/step - loss: 0.4924
Epoch 8/20
20/20 [==============================] - 0s 2ms/step - loss: 0.4001
Epoch 9/20
20/20 [==============================] - 0s 2ms/step - loss: 0.3228
Epoch 10/20
20/20 [==============================] - 0s 2ms/step - loss: 0.2802
Epoch 11/20
20/20 [==============================] - 0s 2ms/step - loss: 0.2724
Epoch 12/20
20/20 [==============================] - 0s 2ms/step - loss: 0.2580
Epoch 13/20
20/20 [==============================] - 0s 2ms/

In [3]:
model.layers

In [4]:
from keras.models import Model

intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.layers[1].output)
intermediate_output = intermediate_layer_model.predict(np.array([x[0]]))

In [5]:
intermediate_output

array([[ 1.145561  , -3.4068832 ,  0.3460977 ,  0.17611848, -2.585769  ,
        -1.9918323 ,  0.39855596, -3.3038175 , -2.2761903 ]],
      dtype=float32)

In [19]:
preds = model.predict(np.array([[0]]))
print(preds[0])

[-0.06594776]


In [7]:
x.shape

TensorShape([2000, 1])

In [9]:
np.array([0]).shape

(1,)

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(x,y)